In [17]:
import open3d as o3d
import numpy as np

import sys
sys.path.insert(0, '../')

from  creation_vector import Vector_Creation
from  Voxel_model import Voxels
from  Path_create import Create_Path
from  mesh_tools import translation
import networkx as nx
from tqdm import tqdm

In [18]:
def visualisation(see_what : list):
    create_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
    see_what.append(create_coordinate_frame)
    o3d.visualization.draw_geometries(see_what, mesh_show_wireframe=False )

In [19]:
tool = o3d.io.read_triangle_mesh('../Object/Deposition Head.stl')
collision = o3d.io.read_triangle_mesh('../Object/collision.stl')

- model voxel

In [20]:
scale_voxel = 0.5
scale_print= 500
number_of_points = 1000000
vox = Voxels(scale_print,scale_voxel)

- object de collision voxel

In [21]:
from Voxelize import Voxelize
from coord_voxel import coord_voxel

colli_voxel = Voxelize(collision , scale_voxel ,number_of_points,True )
colli_voxel_coord = coord_voxel(colli_voxel)

- creation tool_path

In [22]:
path = Create_Path('../Object//UM3E_Copy of Cube Extruder (3).gcode') 
path.read_gcode(12)

Collision = Create_Path('../Object//UM3E_collision.gcode')
Collision.read_gcode(11)
path.centrer(Collision.tool_path)

path.correction(vox.scale_voxel)

#path.figure(colli_voxel_coord)

- centrer le chemin au milieu du model voxel

In [23]:
centre = np.array([colli_voxel_coord[:,0].mean(),colli_voxel_coord[:,1].mean(),0]).astype(int)
colli_voxel_coord = translation(colli_voxel_coord,centre,vox.midle)
path.translat(centre,vox.midle)
#path.figure(colli_voxel_coord)

- centrer le tool

In [24]:
from centrer_mesh0 import centrer_mesh0
centrer_mesh0(tool)

- ajout de densité

In [25]:
vox.add_density(colli_voxel_coord)

vox.density(colli_voxel_coord)

True

- creation vecteur 

In [26]:
from points_Octree import points_Octree
from parcourir_octree_dico import dic

theta , theta_max = 10 , 50
Circle=Vector_Creation(np.deg2rad(theta),np.deg2rad(theta_max))

position_head_layer_circle = {i: [] for i in Circle }
octree_haed_layer_circle = {i: [] for i in Circle }

G = nx.DiGraph()
path_tool = path.tool_path[5000:5001] # you can increase the number of test points on the path
num_layers = len(path_tool)
layer_vertex = {i: [] for i in range(num_layers)}


delta_colli = 4
delta_tool = 4
octree_collision = points_Octree(colli_voxel_coord , delta_colli)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(colli_voxel_coord)

octree_collision_dic = {}
dic(octree_collision,octree_collision_dic)
#visualisation([octree_collision,pcd])

In [27]:
num_layers

1

In [28]:
delta_tool

4

- fonction free_path

In [29]:
from Free_path_octrees import Free_path_octrees

free_path=Free_path_octrees(vox, path_tool, tool , theta, position_head_layer_circle,Circle,G, layer_vertex , octree_haed_layer_circle , octree_collision_dic , delta_tool, number_of_points)

free_path[1:-1]

pcd_tool = o3d.geometry.PointCloud()
pcd_tool.points = o3d.utility.Vector3dVector(position_head_layer_circle[3][3])

visualisation([pcd,pcd_tool])

100%|████████████████████████████████████████████| 1/1 [02:10<00:00, 130.23s/it]


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
